In [286]:
import pandas as pd
data=pd.read_csv('data_zoopla_edinburgh.csv')
data.index.rename('property_index',inplace=True)
data.head()

,title,address,price,description,features,number_beds,number_baths,images,letting_agent_name,available_from
property_index,,,,,,,,,,
0,2 bed flat to rent,"Mayfield Road, Newington, Edinburgh EH9","£1,400 pcm",Fantastic property in Newington comprising two...,"Furnished,Fridge Freezer,Dishwasher,Central He...",2 beds,1 bath,https://lid.zoocdn.com/u/480/360/9eb49fee0d69d...,Grant Property,Available from 25 February 2022
1,1 bed flat to rent,"South Gyle Mains, South Gyle, Edinburgh EH12",£750 pcm,Well presented one double property in popular ...,"Furnished,Central Heating,Double Glazing,No Sm...",1 bed,1 bath,https://lid.zoocdn.com/u/480/360/c808c8247e26a...,Cornerstone Letting,Available from 10 December 2021
2,2 bed flat to rent,"Kimmerghame Terrace, Edinburgh EH4","£1,250 pcm",Scottish Letting Agent Registration Number: LA...,NaN,2 beds,2 baths,https://lid.zoocdn.com/u/480/360/b3101b6e49099...,Your Move - Edinburgh,Available from 20 December 2021
3,1 bed flat to rent,"Wardlaw Street, Gorgie, Edinburgh EH11",£700 pcm,"Tastefully presented ground floor flat, formin...","Furnished,Central Heating,Shower,Double Glazin...",1 bed,1 bath,https://lid.zoocdn.com/u/480/360/1abce30b79f6c...,Marchside Property,Available from 07 January 2022
4,4 bed detached house to rent,"Craigcrook Grove, Blackhall, Edinburgh EH4","£2,995 pcm",This stunning property is located in a quiet c...,"Unfurnished,Central Heating,Shower,Double Glaz...",4 beds,2 baths,https://lid.zoocdn.com/u/480/360/50378a1bbbc6b...,Gregor Fortune Property Ltd,Available from 07 January 2022


In [287]:
data['price']=data['price'].str.replace('[£, pcmw]','')
data['number_beds']=data['number_beds'].str.replace('[beds ]','')
data['number_baths']=data['number_baths'].str.replace('[ baths]','')

data['price']=pd.to_numeric(data['price'])
data['number_beds']=pd.to_numeric(data['number_beds'])
data['number_baths']=pd.to_numeric(data['number_baths'])

C:\Users\usuario\AppData\Local\Temp/ipykernel_9304/814890878.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['price']=data['price'].str.replace('[£, pcmw]','')
C:\Users\usuario\AppData\Local\Temp/ipykernel_9304/814890878.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['number_beds']=data['number_beds'].str.replace('[beds ]','')
C:\Users\usuario\AppData\Local\Temp/ipykernel_9304/814890878.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['number_baths']=data['number_baths'].str.replace('[ baths]','')


In [288]:
data['available_from']=data['available_from'].str.replace(r'(Available from )?','')
data['available_from']=pd.to_datetime(data['available_from'],errors='coerce')
data['available_from'].fillna(pd.to_datetime('today'),inplace=True)


C:\Users\usuario\AppData\Local\Temp/ipykernel_9304/2937408805.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['available_from']=data['available_from'].str.replace(r'(Available from )?','')


In [289]:
data['letting_agent_name'].value_counts().head(20)


DJ Alexander Lettings Ltd                     65
Southside Property Management                 33
Murray & Currie Property                      21
OpenRent                                      16
Rooks Estate Agents & Lettings                15
Houst Lets                                    15
Ben Property Management Ltd                   14
Homelike Internet GmbH                        13
Gilson Gray LLP                               13
Northwood Residential Lettings (Edinburgh)     9
Clan Gordon Ltd                                8
Marchside Property                             7
The Property Letting Centre                    6
Zone Letting Limited                           6
Accommodation.co.uk                            6
Grant Property                                 6
Arden Property Management                      5
Easylet                                        5
Clyde Property, Edinburgh                      5
ACE Property Management                        5
Name: letting_agent_

In [290]:
data['images']=data['images'].str.split(',')


In [291]:
data['images']


property_index
0      [https://lid.zoocdn.com/u/480/360/9eb49fee0d69...
1      [https://lid.zoocdn.com/u/480/360/c808c8247e26...
2      [https://lid.zoocdn.com/u/480/360/b3101b6e4909...
3      [https://lid.zoocdn.com/u/480/360/1abce30b79f6...
4      [https://lid.zoocdn.com/u/480/360/50378a1bbbc6...
                             ...                        
377    [https://lid.zoocdn.com/u/480/360/52e45702a659...
378    [https://lid.zoocdn.com/u/480/360/fe2ff342c581...
379    [https://lid.zoocdn.com/u/480/360/d85e5ff05f18...
380    [https://lid.zoocdn.com/u/480/360/2d56261ea6e2...
381    [https://lid.zoocdn.com/u/480/360/46651bca196e...
Name: images, Length: 382, dtype: object